In [4]:
import pandas as pd

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats



In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px

st.set_page_config(page_title="CaféData", layout="wide")

st.title("☕ CaféData")
st.subheader("Análisis simple para cafeterías y tiendas")

# --- Cargar archivo ---
archivo = st.file_uploader(
    "Sube tu archivo de ventas (Excel)",
    type=["xlsx", "csv"]
)

if archivo is not None:
    df = pd.read_excel(archivo)

    # --- Validación mínima ---
    columnas_requeridas = {
        "fecha", "producto", "categoria",
        "cantidad", "precio_unitario", "costo_unitario"
    }

    if not columnas_requeridas.issubset(df.columns):
        st.error("❌ El archivo no tiene las columnas necesarias")
        st.stop()

    # --- Procesamiento ---
    df["fecha"] = pd.to_datetime(df["fecha"])
    df["ingresos"] = df["cantidad"] * df["precio_unitario"]
    df["costo_total"] = df["cantidad"] * df["costo_unitario"]
    df["ganancia"] = df["ingresos"] - df["costo_total"]

    # --- KPIs ---
    col1, col2, col3 = st.columns(3)
    col1.metric("💰 Ingresos totales", f"${df['ingresos'].sum():,.0f}")
    col2.metric("📦 Ganancia total", f"${df['ganancia'].sum():,.0f}")
    col3.metric("📈 Margen promedio", f"{(df['ganancia'].sum()/df['ingresos'].sum())*100:.1f}%")

    # --- Gráfico 1: Ventas por día ---
    ventas_dia = df.groupby("fecha")["ingresos"].sum().reset_index()
    fig1 = px.line(ventas_dia, x="fecha", y="ingresos", title="Ventas por día")
    st.plotly_chart(fig1, use_container_width=True)

    # --- Gráfico 2: Productos más rentables ---
    rentabilidad = (
        df.groupby("producto")["ganancia"]
        .sum()
        .sort_values(ascending=False)
        .reset_index()
    )
    fig2 = px.bar(
        rentabilidad.head(10),
        x="producto",
        y="ganancia",
        title="Productos más rentables"
    )
    st.plotly_chart(fig2, use_container_width=True)

    # --- Insight automático ---
    top_producto = rentabilidad.iloc[0]
    st.success(
        f"📌 Tu producto más rentable es **{top_producto['producto']}** "
        f"con ${top_producto['ganancia']:,.0f} de ganancia."
    )

else:
    st.info("👆 Sube un archivo para comenzar")
